In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [52]:
dataset = pd.read_csv("../input/vehicle-dataset-from-cardekho/Car details v3.csv")
dataset.head()
dataset.info()

In [53]:
# first need to convert to dataframe then drop multiple columns
df = pd.DataFrame(dataset)
df.drop(df.columns[[0,1,5,6,7,9,11]], axis = 1, inplace = True)
df
df.info()

In [54]:
df.drop(df.columns[[2, 5]], axis = 1, inplace = True)
df.info()

In [55]:
#to separate int and string inside columns
df[['mileage','str']] = df['mileage'].str.split(' ', 1, expand=True)
df.info()

In [56]:
#to separate int and string inside columns
df[['max_power','bhp']] = df['max_power'].str.split(' ', 1, expand=True)
df.info()

In [57]:
df.drop(df.columns[[4, 5]], axis = 1, inplace = True)
df.info()

In [58]:
df[['mileage','max_power']] = df[['mileage','max_power']].apply(pd.to_numeric)
df.info()

In [59]:
df
